In [1]:
from framework import Framework

In [2]:
data_dir = "daily_cluster_data/daily_cluster_matrices_min_6"

fw = Framework(data_dir,verbose=1)

experimenting with:
 ['louvain']


In [4]:
fw.run_experiment(1,30)

creating matrices for sample_0
creating adjacency matrix with cooccurrence 0.04 seconds
creating adjacency matrix with jaccard 17.08 seconds
creating adjacency matrix with dice 16.89 seconds
creating adjacency matrix with cosine_nmf 0.36 seconds
creating adjacency matrix with cosine_svd 0.11 seconds
creating adjacency matrix with correlation 0.01 seconds
creating adjacency matrix with pmi 7.98 seconds
creating adjacency matrix with lift 23.71 seconds
creating adjacency matrix with tfidf Error with tfidf: '1tv'
creating adjacency matrix with conditional 23.72 seconds
creating adjacency matrix with symmetric_conditional 23.47 seconds


AttributeError: 'DataFrame' object has no attribute 'dtype'

In [31]:
from collections import Counter
from itertools import combinations
from partition_metrics import all_metrics

def compare_two_coms(s1,s2):
    coms1 = list(fw.results[f'sample_{s1}']['normalized_counts']['louvain']['louvain_0']['communities'].values())
    coms2 = list(fw.results[f'sample_{s2}']['normalized_counts']['louvain']['louvain_0']['communities'].values())

    # print(coms1)
    r = all_metrics(coms1,coms2,verbose=0)
    print('ari',r['ari'])
    print('nmi',r['nmi'])
    print('ned',r['norm_edit_distance'])
    print()

for s1,s2 in combinations(list(range(5)),2):
    compare_two_coms(s1,s2)
    # print('-'*60)

ari 0.5805243445692884
nmi 0.7084075737327697
ned 0.7551020408163265

ari 0.6871253583528798
nmi 0.7854456928809548
ned 0.8571428571428572

ari 0.5781781781781782
nmi 0.7461249567432026
ned 0.7551020408163265

ari 0.6660470879801735
nmi 0.7639471613626613
ned 0.7755102040816326

ari 0.7167630057803468
nmi 0.7990392175733769
ned 0.8163265306122449

ari 0.49084611578426524
nmi 0.6706424076641104
ned 0.7142857142857143

ari 0.604735883424408
nmi 0.7296449759330995
ned 0.7755102040816326

ari 0.5929561993683922
nmi 0.7535178703396224
ned 0.7346938775510203

ari 0.5335110434120335
nmi 0.6668593506807521
ned 0.7346938775510203

ari 0.5832900545312906
nmi 0.7618525646808638
ned 0.7142857142857143



In [7]:
import pandas as pd

rows = []
for sample_ID, modeling_methods in fw.results.items():
    for modeling_method, comm_methods in modeling_methods.items():
        for comm_method_name, hp_dict in comm_methods.items():
            for hp_ID, result in hp_dict.items():
                row = {
                    'sample_ID': sample_ID,
                    'modeling_method': modeling_method,
                    'comm_method_name': comm_method_name,
                    'hp_ID': hp_ID,
                    **result  # assuming result is a dict
                }
                rows.append(row)

df = pd.DataFrame(rows)
df

,sample_ID,modeling_method,comm_method_name,hp_ID,hyperparameters,communities
0,sample_0,normalized_counts,louvain,louvain_0,"{'parameter1': 5, 'parameter2': 12}","{0: 0, 1: 1, 2: 1, 3: 2, 4: 0, 5: 0, 6: 5, 7: ..."
1,sample_1,normalized_counts,louvain,louvain_0,"{'parameter1': 5, 'parameter2': 12}","{0: 0, 1: 1, 2: 2, 3: 2, 4: 0, 5: 0, 6: 3, 7: ..."
2,sample_2,normalized_counts,louvain,louvain_0,"{'parameter1': 5, 'parameter2': 12}","{0: 0, 1: 1, 2: 2, 3: 2, 4: 0, 5: 0, 6: 3, 7: ..."
3,sample_3,normalized_counts,louvain,louvain_0,"{'parameter1': 5, 'parameter2': 12}","{0: 0, 1: 1, 2: 2, 3: 3, 4: 0, 5: 0, 6: 4, 7: ..."
4,sample_4,normalized_counts,louvain,louvain_0,"{'parameter1': 5, 'parameter2': 12}","{0: 0, 1: 1, 2: 2, 3: 1, 4: 0, 5: 0, 6: 3, 7: ..."
